In [10]:
import setup
setup.init_django()

In [11]:
import helpers.clients as helpers_clients

In [12]:
company_name = "Google"
company_ticker = "GOOG"
multiplier = 1
from_data= "2024-01-09"
to_date = "2024-04-09"

client = helpers_clients.PolygonAPIClient(
    ticker = company_ticker,
    multiplier = multiplier,
    from_data = from_data,
    to_date = to_date,
)
dataset = client.get_stock_data()

In [13]:
len(dataset)

5000

In [14]:
new_stock_data = dataset[0]

In [15]:
new_stock_data

{'open_price': 140.41,
 'close_price': 140.41,
 'high_price': 140.41,
 'low_price': 140.41,
 'volume': 134,
 'number_of_trades': 3,
 'volume_weighted_average': 140.3984,
 'time': datetime.datetime(2024, 1, 9, 9, 2, tzinfo=<UTC>)}

In [16]:
from market.models import Company, StockQuote

In [17]:
company_obj, created = Company.objects.get_or_create(name=company_name, ticker=company_ticker)

In [18]:
company_obj

<Company: Company object (2)>

In [19]:
StockQuote.objects.all()

<QuerySet [<StockQuote: StockQuote object (1)>]>

In [20]:
batch_size = 1000
for i in range(0, len(dataset), batch_size):
    batch_chunk = dataset[i:i+batch_size]
    chunked_quotes = []
    for data in batch_chunk:
        chunked_quotes.append(
            StockQuote(company=company_obj, **data)
        )
    StockQuote.objects.bulk_create(chunked_quotes, ignore_conflicts=True)

In [21]:
StockQuote.objects.all().count()

5001